In [1]:
import sys
sys.path.append('/home/lumargot/trachoma/src/py')

import os 
os.environ["CUDA_VISIBLE_DEVICES"] = "-1" # put -1 to not use any

In [ ]:
import math
import pandas as pd
import numpy as np 
import seaborn as sns
from utils import remove_labels

In [3]:
import SimpleITK as sitk
import torch
import matplotlib.pyplot as plt

In [4]:
# df_train = pd.read_csv('/CMF/data/lumargot/trachoma/mtss_seg.csv')
df_train = pd.read_csv('/CMF/data/lumargot/trachoma/csv_mtss_pret/csv_updated/mtss_pret_combined.csv')
df_train = df_train.loc[df_train['dataset'] !='Pret']

In [5]:
seg_column='seg'
class_column='class'
mount_point='/CMF/data/lumargot/trachoma'
drop_labels=['Reject', 'Short Incision']
concat_labels=['overcorrection', 'Gap', 'ECA', 'Fleshy']
label_column = 'label'

In [7]:
df_train = remove_labels(df_train, class_column, label_column, drop_labels=drop_labels, concat_labels=concat_labels)

In [33]:
df_train['class'] = df_train['class'] -1 
df_train[['class', 'label']].value_counts()

class  label         
0      Healthy           13458
2      overcorrection     2637
1      Entropion          1302
2      Gap                 437
       ECA                 365
       Fleshy              248
Name: count, dtype: int64

In [9]:
df_unique_subjects= df_train.drop_duplicates(subset='filename')

In [10]:
all_filenames = df_unique_subjects['filename'].to_list()

In [36]:
l_left, l_middle, l_right = [], [], []
for filename in all_filenames:

  df_subject = df_train.loc[ df_train['filename'] == filename]

  seg_path = filename.replace('img', 'seg_old').replace('.jpg', '.nrrd')
  seg_path = os.path.join(mount_point, seg_path)
  img_path = os.path.join(mount_point, filename)


  img= sitk.GetArrayFromImage(sitk.ReadImage(img_path)).copy()
  seg_t = torch.tensor(sitk.GetArrayFromImage(sitk.ReadImage(seg_path)).copy())
  seg_t = (seg_t == 3).float()

  bb = compute_eye_bbx(seg_t).numpy()

  seg_cropped = seg_t[bb[1]:bb[3],bb[0]:bb[2]].numpy()
  img_cropped = img[bb[1]:bb[3],bb[0]:bb[2]]

  # plt.figure(figsize=(12,5))
  # plt.subplot(121)
  # plt.imshow(img_cropped)

  # plt.subplot(122)
  # plt.title('cropped seg')
  # plt.imshow(seg_cropped)

  # plt.tight_layout()
  # plt.show()

  x = df_subject['x_patch'] - bb[0]
  y = df_subject['y_patch'] - bb[1]
  labels = df_subject['class']
  h,w = seg_cropped.shape
  
  portion_side = w/4

  outcome_left, outcome_middle, outcome_right = get_outcome_per_section(x, labels, portion_side)
  l_left.append(outcome_left)
  l_middle.append(outcome_middle)
  l_right.append(outcome_right)

[0 2] [3 3]  -- > 2
[0 2] [2 1]  -- > 0
[0 2] [7 2]  -- > 0
[0 2] [5 1]  -- > 0
[0 2] [1 2]  -- > 0
[0 2] [1 5]  -- > 2
[0 2] [3 6]  -- > 2
[0 2] [4 1]  -- > 0
[0 2] [1 1]  -- > 0
[0 2] [1 3]  -- > 2
[0 2] [6 4]  -- > 2
[0 2] [4 1]  -- > 0
[0 1] [2 1]  -- > 0
[0 1] [7 2]  -- > 0
[0 1] [5 2]  -- > 0
[0 2] [5 1]  -- > 0
[0 2] [2 5]  -- > 2
[0 1] [7 3]  -- > 1
[0 2] [3 7]  -- > 2
[0 2] [4 4]  -- > 2
[0 2] [3 2]  -- > 0
[0 2] [3 2]  -- > 0
[0 2] [1 2]  -- > 0
[0 1] [5 1]  -- > 0
[0 1] [2 3]  -- > 1
[0 2] [2 1]  -- > 0
[0 2] [2 1]  -- > 0
[0 2] [5 1]  -- > 0
[0 2] [6 3]  -- > 2
[0 2] [5 3]  -- > 2
[0 1] [3 3]  -- > 1
[0 1] [2 4]  -- > 1
[0 2] [1 3]  -- > 2
[0 1] [3 2]  -- > 0
[0 1] [5 4]  -- > 1
[0 2] [1 4]  -- > 2
[0 2] [5 2]  -- > 0
[0 2] [3 1]  -- > 0
[0 2] [8 1]  -- > 0
[0 2] [3 2]  -- > 0
[0 2] [5 1]  -- > 0
[0 1] [9 3]  -- > 1
[0 2] [4 1]  -- > 0
[0 1] [4 6]  -- > 1
[0 2] [4 1]  -- > 0
[0 2] [2 3]  -- > 2
[0 2] [4 3]  -- > 2
[0 2] [7 3]  -- > 2
[0 1] [3 1]  -- > 0
[0 1] [6 3]  -- > 1


In [37]:
df_unique_subjects['outcome_left'] = l_left
df_unique_subjects['outcome_middle'] = l_middle
df_unique_subjects['outcome_right'] = l_right

/tmp/ipykernel_365214/725881009.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unique_subjects['outcome_left'] = l_left
/tmp/ipykernel_365214/725881009.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unique_subjects['outcome_middle'] = l_middle
/tmp/ipykernel_365214/725881009.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/p

In [38]:
df_unique_subjects

,index,Unnamed: 0.1,Unnamed: 0,cid,x_patch,y_patch,label,filename,dataset,class,to_drop,outcome_left,outcome_middle,outcome_right
0,9103,9113,9113,11010,1114,2426,Healthy,PoPP_Data/mtss/img/11010_12Intra_os_postop.jpg,PoPP_Data,1,0,-1,0,0
12,9115,9137,9137,11006,1715,2171,ECA,PoPP_Data/mtss/img/11006_12Intra_od_postop.jpg,PoPP_Data,3,0,-1,2,2
17,9120,9147,9147,11011,889,2198,Healthy,PoPP_Data/mtss/img/11011_12Intra_os_postop.jpg,PoPP_Data,1,0,-1,0,0
29,9132,9171,9171,11033,1250,3111,Healthy,PoPP_Data/mtss/img/11033_12Intra_od_postop.jpg,PoPP_Data,1,0,-1,0,0
40,9143,9193,9193,11016,1936,2235,Gap,PoPP_Data/mtss/img/11016_12Intra_os_postop.jpg,PoPP_Data,3,0,0,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18385,27488,45929,45929,14113,1364,2460,Healthy,PoPP_Data/mtss/img/14113_12Intra_od_postop.jpg,PoPP_Data,1,0,-1,0,0
18395,27498,45951,45951,14097,912,2192,Entropion,PoPP_Data/mtss/img/14097_12Intra_od_postop.jpg,PoPP_Data,2,0,0,1,0
18409,27512,45979,45979,14078,1153,2553,overcorrection,PoPP_Data/mtss/img/14078_12Intra_od_postop.jpg,PoPP_Data,3,0,0,2,0
18421,27524,46003,46003,14059,903,2678,Entropion,PoPP_Data/mtss/img/14059_12Intra_os_postop.jpg,PoPP_Data,2,0,-1,1,0


In [39]:
df_unique_subjects[['cid', 'filename', 'dataset', 'outcome_left', 'outcome_middle', 'outcome_right']].to_csv('/CMF/data/lumargot/trachoma/PoPP_Data/mtss/outcomes_3_sections_v2.csv')